In [1]:
# !pip install --upgrade faiss-cpu hnswlib

# LSH

[**FALCONN**](https://github.com/FALCONN-LIB/FALCONN) - LSH Families for cosine similarity*
  
  2015 - Practical and Optimal LSH for Angular Distance https://arxiv.org/abs/1509.02897
  
  Туториал по `LSH` и его реализации в библиотеке (оно же, коротко и ясно):
  
  https://github.com/FALCONN-LIB/FALCONN/wiki/LSH-Primer <br>
  https://github.com/FALCONN-LIB/FALCONN/wiki/LSH-Families
  
  
[**datasketch**](https://github.com/ekzhu/datasketch) - LSH Family for Jaccard similarity

#### Сделаем данные

In [2]:
import numpy as np

dim = 128
num_elements = 100_000

# Generating sample data
np.random.seed(911)
data = np.random.random((num_elements, dim)).astype(np.float32)
ids = np.arange(num_elements)

# HNSW - Hierarchical Navigable Small World
  
  [**nmslib**](https://github.com/nmslib/nmslib/)
  
  
  [**hnswlib**](https://github.com/nmslib/hnswlib)
  
**Navigable Small World** <br>
  2013 - Approximate nearest neighbor algorithm based on navigable
small world graphs https://publications.hse.ru/pubs/share/folder/x5p6h7thif/128296059.pdf


**Hierarchical Navigable Small World** <br>
  2016 - Efficient and robust approximate nearest neighbor search using Hierarchical Navigable Small World graphs https://arxiv.org/abs/1603.09320

## Псевдокод
### **Описание шагов псевдокода**:

1. **Случайный уровень**: 
   - Для каждого нового узла случайно выбирается уровень (на котором он будет присутствовать) с использованием **геометрического распределения**.

2. **Точка входа**:
   - Начальная точка (root) для поиска ближайших соседей. Если граф пустой, первый элемент становится точкой входа.

3. **Поиск ближайших соседей**:
   - На каждом уровне выполняется **локальный поиск** ближайших соседей, начиная с текущей точки (точки входа или предыдущих уровней).

4. **Соединение узлов**:
   - Узел связывается с ближайшими соседями на текущем уровне (ограничивается параметром **M**).
   - Обновляются связи соседей, чтобы поддерживать баланс структуры.

5. **Переход между уровнями**:
   - Поиск переходит на более низкий уровень, используя уже найденные узлы для навигации.

6. **Обновление точки входа**:
   - Если новый узел имеет уровень выше текущей точки входа, он становится новой точкой входа.

---

### **Функции в псевдокоде**:

1. `random_level()`:
   - Возвращает случайный уровень на основе распределения. Чем выше уровень, тем меньше вероятность появления узла.

2. `search_layer(current_point, target, level, efConstruction)`:
   - Выполняет локальный поиск ближайших соседей на заданном уровне, начиная с `current_point`.

3. `select_closest_neighbors(neighbors, M)`:
   - Выбирает **M** ближайших соседей из списка.

4. `connect_node(node, neighbors, level)`:
   - Соединяет узел с выбранными соседями на заданном уровне.

5. `update_neighbor_connections(neighbor, node, M, level)`:
   - Обновляет связи для соседей с учётом максимального количества связей.

6. `navigate_to_closest_node(current_point, target, level)`:
   - Выполняет "жадный" поиск ближайшего узла для перехода на более низкий уровень.

---

### **Параметры алгоритма**:
- **M**: Количество связей на уровнях выше 0.
- **M_max**: Количество связей на 0-м уровне (базовом уровне).
- **efConstruction**: Параметр, управляющий точностью поиска ближайших соседей при добавлении элементов.

In [ ]:
function HNSW_Build(data, M, M_max, efConstruction):
    # data: массив объектов (векторов), которые нужно добавить в граф
    # M: максимальное количество связей (neighbors) на одном уровне
    # M_max: максимальное количество связей на 0-ом уровне
    # efConstruction: фактор поиска при построении (контролирует точность поиска ближайших соседей)
    
    graph = initialize_empty_graph()  # Пустая структура графа
    enter_point = None               # Точка входа (первый добавленный элемент)
    
    for each point in data:          # Итерация по всем объектам в данных
        level = random_level()       # Определить случайный уровень для текущего объекта
        node = create_node(point, level)
        
        if enter_point is None:      # Если граф пустой — установить первую точку
            enter_point = node
            add_node_to_graph(graph, node)
            continue
        
        current_point = enter_point  # Начнем с точки входа для поиска ближайших соседей
        
        # Добавление элемента на каждом уровне
        for current_level from max_level to 0:  # Идем сверху вниз
            if current_level <= level:
                neighbors = search_layer(current_point, node, current_level, efConstruction)
                neighbors = select_closest_neighbors(neighbors, M)  # Ограничить количеством M
                
                connect_node(node, neighbors, current_level)  # Соединить текущий узел с соседями
                
                # Обновить связи соседей, если это необходимо
                for neighbor in neighbors:
                    update_neighbor_connections(neighbor, node, M, current_level)
            
            # Перейти к более низкому уровню и найти ближайший узел
            if current_level > 0:
                current_point = navigate_to_closest_node(current_point, node, current_level)
        
        # Обновить глобальную точку входа, если уровень нового элемента выше текущей точки входа
        if level > enter_point.level:
            enter_point = node
        
        add_node_to_graph(graph, node)
    
    return graph

In [3]:
import hnswlib
from time import time as tm


# Declaring index
p = hnswlib.Index(space='l2', dim=dim) # possible options are l2, cosine or ip

# Initializing index - the maximum number of elements should be known beforehand
p.init_index(max_elements=num_elements, ef_construction=200, M=16)


stm = tm()

# Element insertion (can be called several times):
p.add_items(data, ids)

print('Element insertion:', round(tm() - stm, 3))




# Controlling the recall by setting ef:
p.set_ef(50) # ef should always be > k

query_size = 1000
query = data[:query_size]

stm = tm()
# Query dataset, k - number of closest elements (returns 2 numpy arrays)
labels, distances = p.knn_query(query, k=3)

print('Query dataset:', round(tm() - stm, 3))

print('\nlabels:\n', labels[:5])
print('\ndistances:\n', distances[:5])

Element insertion: 4.769
Query dataset: 0.019

labels:
 [[    0  1282 57404]
 [    1 51155 44628]
 [26825 49496 40012]
 [    3 59762 46046]
 [    4 90650 90863]]

distances:
 [[ 0.        12.8320055 13.142619 ]
 [ 0.        11.866396  12.505065 ]
 [14.063374  14.3945465 15.148863 ]
 [ 0.        13.108384  13.354358 ]
 [ 0.        13.288245  13.621016 ]]


Описание параметров с рекомендациями - https://github.com/nmslib/hnswlib/blob/master/ALGO_PARAMS.md

  Рекомендации также можно найти в основной статье

# Faiss - Facebook AI Research Similarity Search
  
  [**faiss**](https://github.com/facebookresearch/faiss) <br>
  
  2011- Product Quantization for Nearest Neighbor Search https://hal.inria.fr/inria-00514462v2/document

In [4]:
import faiss

# build the index
coarse_quantizer = faiss.IndexFlatL2(dim)

index = faiss.IndexIVFPQ(coarse_quantizer, # с помощью какого индекса считать соседство с центроидами (coarse и PQ)
                         dim, # d - размерность исходных векторов
                         1000, # nlists - k' для coarse quantizer
                         16, # m - на сколько векторов бить исходные в product quantizer
                         8 # nbits - количество бит на индексы центроидов для PQ, nbits = log_2 k*, то есть k* = 2^nbits
                        )

print('index.is_trained:', index.is_trained)

train_data = data.copy() # на практике это могут быть другие данные


stm = tm()
index.train(data)
print('Training:', round(tm() - stm, 3))


stm = tm()
index.add(data) # add vectors to the index
print('Adding data to index:', round(tm() - stm, 3))

k = 3

stm = tm()
distances, labels = index.search(query, k)
print('Searching:', round(tm() - stm, 3))

print('\nlabels:\n', labels[:5])
print('\ndistances:\n', distances[:5])

index.is_trained: False
Training: 7.122
Adding data to index: 0.247
Searching: 0.003

labels:
 [[    0 61766 21369]
 [    1 68825 13730]
 [    2 35009 50373]
 [    3 20159 63718]
 [    4  2549 42519]]

distances:
 [[ 2.6053514 11.897097  13.493942 ]
 [ 2.4028902 11.932099  12.539666 ]
 [ 2.6983104 13.666088  13.668032 ]
 [ 2.0409832 12.788938  13.140978 ]
 [ 2.9106393 12.583055  14.28035  ]]


Рекомендации по параметрам ищите в статьях или на wiki

Сравнения :)

In [6]:
_ = '''
HNSW:
Element insertion: 4.769
Searching: 0.019

Faiss:
Training: 7.122
Element insertion: 0.247
Searching: 0.003
'''

# *Вместо заключения

- бенчмарки алгоритмов ANN - http://ann-benchmarks.com/index.html
- [google ScaNN](https://github.com/google-research/google-research/tree/master/scann) - вроде рулит, но без документации
- [PyNNDescent](https://github.com/lmcinnes/pynndescent) - user-defined distances